 <table style="background-color: white;width:100%;">
    <tr style="display:none">
        <td></td>
        <td></td>
    </tr>
    <tr style="height:3em">
        <td style="width:20%;align:left"><img src="iut_bordeaux.jpg"></td>
        <td style="width:60%;font: bold 2em 'Fira Sans', serif;text-align:center"> TP  2<br>Euclide et exponentiation modulaire</td>
        <td style="width:20%;font: bold 1.3em 'Fira Sans', serif;vertical-align:top;">R01.06<i></i></td>
    </tr>
</table>

## Exponentiation modulaire

Nous avons souvent besoin de calculer rapidement des puissances modulo $n$. Pour cela il existe une méthode beaucoup plus efficace que de calculer d’abord $a^k$ puis de le réduire modulo $n$. Il faut garder à l’esprit que les entiers manipulés ont parfois des dizaines voire des centaines de chiffres.

Commençons par donner une fonction naïve de calcul d'exponentiation modulaire. Elle sera utilisée à la fin de ce TP.

In [16]:
def naive(n,p,m):
    return (n**p)%m

#### Un premier exemple d'exponentiation rapide: $5^{11} (14)$

L’idée est de seulement calculer $5,5^2,5^4,5^8$ ...et de réduire modulo $n$ à chaque fois. 

Pour cela on remarque que $11 = 2^3+2^1+2^0=8 + 2 + 1$ en décomposant uniquement avec des puissances de 2, donc
on calcule les $5^{2^i}(14)$ :

$5\equiv5 (14)$

$5^2 \equiv 25\equiv 11 (14)$

$5^4 \equiv 5^2\times 5^2 \equiv 11×11\equiv 121\equiv 9 (14) $

$5^8 \equiv 5^4 \times 5^4 \equiv 9×9\equiv 81\equiv 11 (14)$

A chaque étape est effectuée une multiplication modulaire. 

Conséquence :
$5^{11} \equiv 5^{8+2+1}(14)\equiv5^8 \times 5^2 \times 5(14)$

$5^{11} \equiv 11 \times 11 \times 5 (14)\equiv 11 \times 55 \equiv 11 \times 13 \equiv 143 \equiv 3(14).$

Nous obtenons donc un calcul de $5^{11} ( 14)$ en 5 opérations au lieu de $10$ si on avait fait 5×5×5... et surtout avec des nombres plus petits à manipuler.

En fait, cette technique est basée sur la décomposition en base 2 de l'exposant.

Voici un autre exemple à compléter manuellement **dans ce fichier** :

Calculer $17^{154} (100)$. 

Tout d’abord on décompose l’exposant k = 154 en base 2 : 

> $(154)_{10}=(10011010)_2$

On en déduit la décompostion en somme de puissance de 2.

> $154 =2^7+2^4+2^3+2$

Ensuite on calcule $17,17^2,17^4,17^8,... ( 100)$

>$$
\begin{align*}
17 & \equiv  17[100]\\
17^2 &\equiv  17\times 17 \equiv  89[100]\\
17^4 & \equiv 89 \times 89 \equiv 21[100]\\
17^8  & \equiv 41[100]\\
17^{16} & \equiv 81[100]\\
17^{32} & \equiv 61[100]\\
17^{64}  & \equiv 21[100]\\
17^{128} & \equiv 41[100]\\
\end{align*}
$$

Il ne reste qu’à rassembler :

> $17^{154} \equiv 17^{2^7+2^4+2^3+2}\equiv 41\times 81\times 41 \times 89 \equiv 29[100] $

1. Ecrire une fonction `expo` qui prend en paramètre trois entier $p,n,m$ et renvoie une liste contenant les $p^{2^i}[m]$ pour $i$ entre $0$ et $n$.  
Elle utilisera obligatoirement la technique vue ci-dessus.

In [6]:
def expo(p,n,m):
    liste=[]
    liste.append(p%m)
    for puis in range(1,n+1):
        liste.append((liste[puis-1]*liste[puis-1])%m);
    return liste

print(expo(5,3,14))
print(expo(17,7,100))

[5, 11, 9, 11]
[17, 89, 21, 41, 81, 61, 21, 41]


21

2. Ecrire une fonction `binaire` qui prend comme paramètre un nombre en décimal et retourne sa décomposition en binaire (sous forme de liste)

In [2]:
def binaire(p):
    liste=[]
    while p>0:
        p,r=p//2,p%2
        liste.append(r)
    liste.reverse()
    return liste

binaire(154)

[1, 0, 0, 1, 1, 0, 1, 0]

3. Ecrire une fonction `exponentiation_rapide(p,n,m)` qui retourne $p^n[m]$

In [14]:
def exponentiation_rapide(p,n,m):
    binaire_expo=binaire(n)
    binaire_expo.reverse()
    puis=expo(p,len(binaire(n)),m)
    result=1;
    for pu in range(len(binaire_expo)) :
        if binaire_expo[pu]==1:
            result*=(puis[pu])%m
    
    return result%m

print(exponentiation_rapide(5,11,14))
print(exponentiation_rapide(17,154,100))

3
29


Python a déjà une fonction `pow` qui permet le calcul d'une exponentiation modulaire.

Comparer le temps de calcul de votre fonction , de `pow` et de la fonction naïve pour :

```python
n=24433  
p=123121  
m=72 
```


In [17]:
import time

n=24433
p=123121
m=72

start = time.time()
pow(n,p,m)
end = time.time()
elapsed = end - start
print(f'Temps d\'exécution pour pow: {elapsed:.4}ms') 


start = time.time()
exponentiation_rapide(n,p,m)
end = time.time()
elapsed = end - start
print(f'Temps d\'exécution pour exponentiation_rapide: {elapsed:.4}ms')  

start = time.time()
naive(n,p,m)
end = time.time()
elapsed = end - start
print(f'Temps d\'exécution pour naive: {elapsed:.4}ms')  

Temps d'exécution pour pow: 5.293e-05ms
Temps d'exécution pour exponentiation_rapide: 8.488e-05ms
Temps d'exécution pour naive: 0.1106ms


## Algorithme d'Euclide

On rappelle l’algorithme d’Euclide :

Soit $a$ et $b$ deux entiers naturels dont l’un au moins est non nul. Le PGCD des nombres a et b peut être obtenu par l’algorithme suivant :

* On pose $r_0 = a, r_1 = b \text{ et } k = 1$
* Tant que $r_k \neq 0$
    On effectue la division euclidienne de $r_{k−1}$ par $r_k$.  
    En notant $q_k$ le quotient et $r_{k+1}$
le reste, on obtient :
$$r_{k−1} = r_k\times q_k + r_{k+1} \text{ avec } 0 \leq  r_{k+1} < r_k$$
* On incrémente $k$ de $1$.

En effectuant cet algorithme, il existera toujours $n \geq 0$ tel que $r_{n+1} = 0$, ce qui assure que
l’algorithme s’arrête. 

On a alors $PGCD(a,b) = r_n$.


1 . Créer une fonction PGCD prenant en entrée deux entiers naturels $a$ et $b$ et qui renvoie $PGCD(a,b)$  en procédant comme l’algorithme d’Euclide.
* On utilisera une liste R contenant les valeurs des $r_k$.
* On affichera les étapes de manière à pouvoir comparer à une exécution “à la main” de l’algorithme. 

In [22]:
def PGCD(a,b,trace=False):
    R=[]
    k=1
    R.append(a)
    R.append(b)
    while R[k] != 0:
        q=R[k-1]//R[k]
        R.append(R[k-1]%R[k])
        if trace:
            print(f"{R[k-1]}={R[k]}*{q}+{R[k+1]}")
        k+=1
    return R[k-1]

PGCD(139,17,True)
    

139=17*8+3
17=3*5+2
3=2*1+1
2=1*2+0


1

2 . Créer une fonction $PPCM$ qui calcule le $PPCM$ de deux entiers non nuls $a$ et $b$.

In [23]:
def PPCM(a,b):
    
    return (a*b)//PGCD(a,b)

PPCM(12,5)

60

3 . Déterminer les couples $(a, b)$ d’entiers naturels $a \leq  b$ tels que $a + b = 256$ et $PGCD(a,b) = 16$

Retouver ces couples par une démonstration.

In [24]:
L=[]
for a in range(0,257):
    b=256-a
    if a<=b and PGCD(a,b)==16:
        L.append({a,b})
print(L)

[{16, 240}, {48, 208}, {80, 176}, {112, 144}]


votre démonstration ici



4 . Résoudre dans $\mathbb{N^2}$

$
PPCM(x,y)+11\times PGCD(x,y)=203
$

In [28]:
L=[]
for x in range(1,203):
    for y in range(1,203):
        if PPCM(x,y)+11*PGCD(x,y)==203:
            if(x<=y) :
                L.append({x,y})
print(L)

[{192, 1}, {64, 3}, {126, 7}, {14, 63}]


## Test de primalité

On donne le petit **théorème de fermat :** 

>si p est un nombre premier alors pour tout $a \in \mathbb{N}, 
\quad 1\leq a <p, \quad a^{p-1}\equiv 1[p]$

Vérifier ce théorème pour les nombres premiers inférieurs à 100 . vous pourrez reprendre votre fonction `Eratosthene` du TP 1.


In [31]:
from math import sqrt

def erast(N):
    #L=list(range(N+1))
    #for i in L:
    #   L[i]=True
        
    L=[True for i in range(N+1)]
    L[0],L[1]=False,False

    for i in range(int(sqrt(N)+1)):
        if L[i]:
            j=2*i
            while j<=N:
                if L[j]:
                    L[j]=False
                j=j+i
    
    return [i for i,e in enumerate(L) if e]

def test_fermat():
    P=erast(100)
    for n in P:
        print(f"pour {n} : ",end=" ")
        for a in range(1,n):
                print(pow(a,p-1,p),end=',')
        print()

test_fermat()

pour 2 :  1,
pour 3 :  1,1,
pour 5 :  1,1,1,1,
pour 7 :  1,1,1,1,1,1,
pour 11 :  1,1,1,1,1,1,1,1,1,1,
pour 13 :  1,1,1,1,1,1,1,1,1,1,1,1,
pour 17 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
pour 19 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
pour 23 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
pour 29 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
pour 31 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
pour 37 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
pour 41 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
pour 43 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
pour 47 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
pour 53 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
pour 59 :  1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1

Par contre, si on fixe $a$, il se peut que $ a^{p-1}\equiv 1[ p]$ sans que $p$ ne soit premier; un tel nombre est dit *pseudo-premier de Fermat* de base $a$. 

Vérifier cette afirmation pour $a=2$ et $p=561=3\times 11 \times 17$

In [32]:
pow(2,560,561)

1

On déduit de ce théorème le test de primalité de Fermat qui repose sur l'idée suivante  : 

>si $p$ est composé (non premier), alors il est *peu probable* que $a^{p–1}$ soit congru à 1 modulo $p$ pour une valeur arbitraire de a. 

On parle alors de nombre *pseudo-premiers*

> En utilisant des tirages aléatoires, donner un entier *pseudo-premier* à 6 chiffres.

On prendra $a=2$ ou $a=3$ pour les tests.

Vérifier que le nombre *pseudo-premier* est ou n'est pas premier avec le théorème de fermat.

In [39]:
from random import randint

def check_pseudo_primal(p):
    return pow(2,p-1,p)==1

def test(p):
    for a in range(1,p):
        if pow(a,p-1,p)  !=1 :
            print(f"test non passé pour a={a}, on a {a}^{p-1}={pow(a,p-1,p)} mod({p})")
            return False
    return True

print(test(5394826801))

t=False
nb_essai=0
while (not t and nb_essai<1000):
    alea=randint(10**5, 10**6-1)
    nb_essai+=1
    if check_pseudo_primal(alea):
        t= not t
        print(f"nombre essais nécessaires :  {nb_essai} et nombre pseudo premier trouvé : {alea}")
if nb_essai<1000 :
    if test(alea):
        print(f" {alea} est un nombre premier")
    else :
        print(f" {alea} n'est pas un nombre premier")

test non passé pour a=7, on a 7^5394826800=4624137259 mod(5394826801)
False
nombre essais nécéssaires :  3 et nombre pseudo premier trouvé : 716951
 716951 est un nombre premier
